In [10]:
import torch 
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from pathlib import Path
import csv
import numpy as np
import cv2
from tqdm import tqdm

from edge_detection import edgeDetection
from interest_point_detection import display_interest_points_matrix
from detect_laser import *
from torchinfo import summary
from torchmetrics.classification import BinaryAccuracy

import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count

In [2]:
# Prepare data 

data_path = Path("./data")
csv_list = list(data_path.glob("*.csv"))
file_map = dict()
test_map = dict()
i = 0
l = 0
for csv_file in csv_list:   
    with open(csv_file, newline='') as csvfile:

        reader = csv.reader(csvfile, delimiter=',')

        header = next(reader)

        # First 2 of every file will be for testing
        for k in range(2):
            row = next(reader)
            if len(row) > 0:
                test_map[l] = dict()
                test_map[l]["name"] = row[0]
                
                for j in range(1, 3):
                    test_map[l][header[j]] = row[j]

                l+=1
        
        for row in reader:
        
            if len(row) > 0:
                file_map[i] = dict()
                file_map[i]["name"] = row[0]
                
                for j in range(1, 3):
                    file_map[i][header[j]] = row[j]

                i+=1

In [10]:
print(len(file_map))
print(len(test_map))

326
24


In [6]:
data_map = dict()
test_data_map = dict()
index = 0
index_test = 0

In [7]:
# Getting images with laser inside them for training
for i in tqdm(file_map):
    #img = cv2.imread(file_map[i]["name"])

    # if (i == 1):
    #     break
    val = 20
    count=0

    for k in range(5,val):
        for j in range(5,val):
            laser_np = (int(file_map[i]["laser.x"]), int(file_map[i]["laser.y"]))

            #temp = img[laser_np[1]-val+k:laser_np[1]+k,laser_np[0]-val+j:laser_np[0]+j]
            # exit(1)
            count+=1

            # Add to data_map
            # 0 is no laser, 1 is yes laser
            data_map[index] = dict()
            data_map[index]["file_name"] = file_map[i]["name"]
            data_map[index]["image"] = (laser_np[1]-val+k, laser_np[0]-val+j)
            data_map[index]["label"] = 1
            index += 1

            # cv2.namedWindow("burger")
            # cv2.imshow("burger", temp)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
    print(count)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 326/326 [00:00<00:00, 1559.09it/s]

225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225
225


In [4]:
# Getting images without any laser for training
for i in tqdm(file_map):
    if i < 45:
        continue
    img = cv2.imread(file_map[i]["name"])
    val=20
    laser_np = (int(file_map[i]["laser.x"]), int(file_map[i]["laser.y"]))
    left = laser_np[0] - 20*2 + 5
    right = laser_np[0] + 35
    top = laser_np[1]-20*2+5
    bottom = laser_np[1]+35

    count = 0
    
    for k in range(50,3000-val-50,85):
        for j in range(50,4000-val-50,85):

            if j >= left and j < right and k >= top and k < bottom:
                continue
            cv2.rectangle(img, (j,k), (j+val, k+val), color=(0,0,255), thickness=10)

            data_map[index] = dict()
            data_map[index]["file_name"] = file_map[i]["name"]
            data_map[index]["image"] = (k, j)
            data_map[index]["label"] = 0
            index += 1
            count+=1
            # temp = img[k:k+val, j:j+val]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 326/326 [00:51<00:00,  6.33it/s]


In [6]:
# Getting images with laser inside them for training
for i in tqdm(test_map):
    # #img = cv2.imread(file_map[i]["name"])
    # if i == 1:
    #     break

    val = 20
    count=0

    for k in range(5,val):
        for j in range(5,val):
            laser_np = (int(test_map[i]["laser.x"]), int(test_map[i]["laser.y"]))

            #temp = img[laser_np[1]-val+k:laser_np[1]+k,laser_np[0]-val+j:laser_np[0]+j]
            # exit(1)
            count+=1

            # Add to data_map
            # 0 is no laser, 1 is yes laser
            test_data_map[index_test] = dict()
            test_data_map[index_test]["file_name"] = test_map[i]["name"]
            test_data_map[index_test]["image"] = (laser_np[1]-val+k, laser_np[0]-val+j)
            test_data_map[index_test]["label"] = 1
            index_test += 1

            # cv2.namedWindow("burger")
            # cv2.imshow("burger", temp)
            # cv2.waitKey(0)
            # cv2.destroyAllWindows()
print(count)

100%|█████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 3151.05it/s]

225


In [7]:
# Getting images without any laser for testing
for i in tqdm(test_map):
  #  img = cv2.imread(test_map[i]["name"])
    # if i == 1:
    #     break
    
    val=20
    laser_np = (int(test_map[i]["laser.x"]), int(test_map[i]["laser.y"]))
    left = laser_np[0] - 20*2 + 5
    right = laser_np[0] + 35
    top = laser_np[1]-20*2+5
    bottom = laser_np[1]+35
    
    for k in range(50,3000-val-50,85):
        for j in range(50,4000-val-50,85):

            if j >= left and j < right and k >= top and k < bottom:
                continue
            # cv2.rectangle(img, (j,k), (j+val, k+val), color=(0,0,255), thickness=2)

            test_data_map[index_test] = dict()
            test_data_map[index_test]["file_name"] = test_map[i]["name"]
            test_data_map[index_test]["image"] = (k, j)
            test_data_map[index_test]["label"] = 0
            index_test += 1
            # temp = img[k:k+val, j:j+val]

100%|██████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<00:00, 817.76it/s]


In [8]:
# data = np.empty((len(data_map), 1200))
# test = np.empty((len(test_data_map), 1200))

In [9]:
# for idx in tqdm(data_map):
#     file_name = data_map[idx]["file_name"]
#     k,j = data_map[idx]["image"]
#     label = data_map[idx]["label"]
    
#     img = cv2.imread(file_name)
#     img_clone = img.copy()
#     cropped_img = img_clone[k:k+20, j:j+20]
#     output_img = np.reshape(cropped_img, (3, 20, 20))

#     data[idx,:] = output_img.flatten()

In [19]:
class LaserDetectionDataset(Dataset):
    """Laser Detection dataset."""

    def __init__(self, data_map, test):
        """
        Arguments:
            data_map: map to training data.
            data: training data
        """
        self.data_map = data_map
        if test:
            self.directory = Path("/home/vsuresh/fishsense-lite-python-pipeline/data/test_binary")
        else:
            self.directory = Path("/home/vsuresh/fishsense-lite-python-pipeline/data/train_binary")

    def __len__(self):

        return len(self.data_map)

    def __getitem__(self, idx):
   
        file = self.directory / f"{idx}.png"
        file_name = file.as_posix()
        k,j = self.data_map[idx]["image"]
        label = self.data_map[idx]["label"]

        # print("Beginning to read image")
        cropped_img = cv2.imread(file_name)
        # print("Finishing reading image")
        # img_clone = img.copy()
        # cropped_img = img_clone[k:k+20, j:j+20]
        output_img = np.reshape(cropped_img, (3, 20, 20))
        # cv2.namedWindow("burger")
        # cv2.imshow("burger", output_img)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        # output_img = np.reshape(self.data[idx,:], (3, 20, 20))
        # print(file_name)
        return output_img.astype(np.float32), np.array([label]).astype(np.float32)

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=(3,3), stride=1, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
        self.pool1 = nn.AvgPool2d(kernel_size=(2, 2))
        
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(3200, 10)
        self.act2 = nn.ReLU()

        self.linear2 = nn.Linear(10, 1)
        
    def forward(self, x):
        x = self.act1(self.conv1(x))
        x = self.pool1(x)
        x = self.drop1(x)
        
        x = self.flatten(x)
        x = self.act2(self.linear1(x))

        x = self.linear2(x)
        
        return x

In [12]:
model = Net()
batch_size = 1
summary(model, input_size=(1, 3, 20, 20), device="cuda")

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [1, 1]                    --
├─Conv2d: 1-1                            [1, 32, 20, 20]           896
├─ReLU: 1-2                              [1, 32, 20, 20]           --
├─AvgPool2d: 1-3                         [1, 32, 10, 10]           --
├─Dropout: 1-4                           [1, 32, 10, 10]           --
├─Flatten: 1-5                           [1, 3200]                 --
├─Linear: 1-6                            [1, 10]                   32,010
├─ReLU: 1-7                              [1, 10]                   --
├─Linear: 1-8                            [1, 1]                    11
Total params: 32,917
Trainable params: 32,917
Non-trainable params: 0
Total mult-adds (M): 0.39
Input size (MB): 0.00
Forward/backward pass size (MB): 0.10
Params size (MB): 0.13
Estimated Total Size (MB): 0.24

In [38]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

laser_dataset = LaserDetectionDataset(data_map = data_map, test=False)
laser_dataloader = DataLoader(laser_dataset, batch_size=2000, shuffle=True, num_workers=12)

test_dataset = LaserDetectionDataset(data_map = test_data_map, test=True)
test_dataloader = DataLoader(test_dataset, batch_size=2000, shuffle=True, num_workers=12)

metric = BinaryAccuracy().to('cuda')
test_metric = BinaryAccuracy().to('cuda')

load_path = Path("model_binary199.pth")
model.load_state_dict(torch.load(load_path.as_posix()))

<All keys matched successfully>

In [40]:
n_epochs = 500

for epoch in range(300, n_epochs):

    for inputs, labels in tqdm(laser_dataloader):

        inputs = inputs.to('cuda')
        labels = labels.to('cuda')
        
        y_pred = model(inputs)

        metric.update(y_pred, labels)

        
        loss = loss_fn(y_pred, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    for test_inputs, test_labels in tqdm(test_dataloader):
        model.eval()
        test_inputs = test_inputs.to('cuda')
        test_labels = test_labels.to('cuda')
        test_pred = model(test_inputs)
        model.train()
        
        test_metric.update(test_pred, test_labels)
                
    print(f"Epoch {epoch}: training accuracy: {metric.compute()}")
    print(f"Epoch {epoch}: test accuracy: {test_metric.compute()}")

    if ((epoch+1) % 10 == 0):
        torch.save(model.state_dict(), f"model_binary{epoch}.pth")

100%|███████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 22.67it/s]


Epoch 300: training accuracy: 0.993766725063324
Epoch 300: test accuracy: 0.9868381023406982


100%|███████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 22.87it/s]


Epoch 301: training accuracy: 0.9937935471534729
Epoch 301: test accuracy: 0.9868881106376648


100%|███████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:00<00:00, 23.02it/s]


...

Epoch 469: training accuracy: 0.9959606528282166
Epoch 469: test accuracy: 0.9901309609413147


100%|███████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:01<00:00, 21.89it/s]


Epoch 470: training accuracy: 0.9959676861763
Epoch 470: test accuracy: 0.9901435375213623


 32%|███████████████████████████▍                                                          | 93/292 [00:02<00:05, 36.46it/s]


KeyboardInterrupt: 

In [15]:
# # laser_dataset = LaserDetectionDataset(data_map = data_map)
# # laser_dataloader = DataLoader(laser_dataset, shuffle=False)

# for i, (img, label) in tqdm(enumerate(laser_dataloader)):

#     display = np.reshape(img.numpy().astype(np.uint8), (20,20,3))
#     print(display.shape)
#     # cv2.namedWindow("burger")
#     # cv2.imshow("burger", display)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
    
#     if i == 20:
#         # cv2.destroyAllWindows()
#         break

In [16]:
# for idx in tqdm(data_map):
#     if idx <= 1965:
#         continue
#     file_name = data_map[idx]["file_name"]
#     k,j = data_map[idx]["image"]
#     label = data_map[idx]["label"]

#     img = cv2.imread(file_name)
#     img_clone = img.copy()
#     cropped_img = img_clone[k:k+20, j:j+20]

#     new_name = f"{idx}.png"
#     save_directory = Path("/home/vsuresh/fishsense-lite-python-pipeline/data/train_binary")

#     save_path = save_directory / new_name
#     # print(save_path.as_posix())
#     cv2.imwrite(save_path.as_posix(), cropped_img)

In [ ]:
# for idx in tqdm(test_data_map):

#     file_name = test_data_map[idx]["file_name"]
#     k,j = test_data_map[idx]["image"]
#     label = test_data_map[idx]["label"]

#     # print("Beginning to read image")
#     img = cv2.imread(file_name)
#     # print("Finishing reading image")
#     img_clone = img.copy()
#     cropped_img = img_clone[k:k+20, j:j+20]
#     # output_img = np.reshape(cropped_img, (3, 20, 20))
#     # cv2.namedWindow("burger")
#     # cv2.imshow("burger", output_img)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
#     # output_img = np.reshape(self.data[idx,:], (3, 20, 20))

#     new_name = f"{idx}.png"
#     save_directory = Path("/home/vsuresh/fishsense-lite-python-pipeline/data/test_binary")

#     save_path = save_directory / new_name
#     # print(save_path.as_posix())
#     cv2.imwrite(save_path.as_posix(), cropped_img)

In [18]:
# def do_work(args):
#     file_name, (k, j), label, idx = args

#     img = cv2.imread(file_name)
#     img_clone = img.copy()
#     cropped_img = img_clone[k:k+20, j:j+20]

#     new_name = f"{idx}.png"
#     save_directory = Path("/home/vsuresh/fishsense-lite-python-pipeline/data/train_binary")

#     save_path = save_directory / new_name
#     # print(save_path.as_posix())
#     cv2.imwrite(save_path.as_posix(), cropped_img)

# list(tqdm(Pool(processes=cpu_count()).imap(do_work, [(data_map[idx]["file_name"], data_map[idx]["image"], data_map[idx]["label"], idx) for idx in data_map])))

In [17]:
# def do_test(args):
#     file_name, (k, j), label, idx = args

#     img = cv2.imread(file_name)
#     img_clone = img.copy()
#     cropped_img = img_clone[k:k+20, j:j+20]

#     new_name = f"{idx}.png"
#     save_directory = Path("/home/vsuresh/fishsense-lite-python-pipeline/data/test_binary")

#     save_path = save_directory / new_name
#     # print(save_path.as_posix())
#     cv2.imwrite(save_path.as_posix(), cropped_img)

# list(tqdm(Pool(processes=cpu_count()).imap(do_test, [(test_data_map[idx]["file_name"], test_data_map[idx]["image"], test_data_map[idx]["label"], idx) for idx in test_data_map])))

In [8]:
print(len(data_map))

73350
